让我们先思考一个问题：对神经网络模型添加新的层，充分训练后的模型**是否只可能更有效地降低训练误差？**理论上，原模型解的空间只是新模型解的空间的子空间。也就是说，如果我们能将新添加的层训练成恒等映射$f(x) = x$，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，**添加过多的层后训练误差往往不降反升**。即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。针对这一问题，何恺明等人提出了**残差网络**（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并**深刻影响了后来的深度神经网络的设计**。

## 5.11.1 残差块

让我们聚焦于神经网络局部。如图5.9所示，设输入为$\boldsymbol{x}$。假设我们希望学出的理想映射为$f(\boldsymbol{x})$，从而作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射$f(\boldsymbol{x})$，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射$f(\boldsymbol{x})-\boldsymbol{x}$。残差映射在实际中往往更容易优化。以本节开头提到的恒等映射作为我们希望学出的理想映射$f(\boldsymbol{x})$。**我们只需将图5.9中右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0**，那么$f(\boldsymbol{x})$即为恒等映射。实际中，当理想映射$f(\boldsymbol{x})$极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。图5.9右图也是ResNet的基础块，即残差块（residual block）。**在残差块中，输入可通过跨层的数据线路更快地向前传播**。



<div align=center>
<img width="400" src="../img/chapter05/5.11_residual-block.svg"/>
</div>
<div align=center>图5.9 普通的网络结构（左）与加入残差连接的网络结构（右）</div>

ResNet沿用了VGG全$3\times 3$卷积层的设计。残差块里首先有2个有相同输出通道数的$3\times 3$卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。**这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加**。如果想改变通道数，就需要引入一个额外的$1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。

In [1]:
import tensorflow as tf
print(tf.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

2.2.0


残差块的实现如下。**它可以设定输出通道数**、**是否使用额外的$1\times 1$卷积层来修改通道数**以及**卷积层的步幅**。

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, activations
class Residual(tf.keras.Model):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual,self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(num_channels,
                                  padding='same',
                                  kernel_size=3,
                                  strides=strides)
        self.conv2 = layers.Conv2D(num_channels,
                                   padding='same',
                                  kernel_size=3)
        if use_1x1conv:
            self.conv3 = layers.Conv2D(num_channels, kernel_size=1, strides=strides)
        else:
            self.conv3 = None
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        
    def call(self,X):
        Y = activations.relu(self.bn1(self.conv1(X)))  #卷积，批量归一，relu激活
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return activations.relu(Y+X)

下面我们来查看输入和输出形状一致的情况。

In [3]:
blk = Residual(3)   # 这里的 3 是什么意思   是通道数
#tensorflow input shpe     (n_images, x_shape, y_shape, channels).
#mxnet.gluon.nn.conv_layers    (batch_size, in_channels, height, width) 
X = tf.random.uniform((4, 6, 6 , 3))
blk(X).shape#TensorShape([4, 6, 6, 3])

TensorShape([4, 6, 6, 3])

我们也可以在增加输出通道数的同时减半输出的高和宽。

In [6]:
blk = Residual(6, use_1x1conv=True, strides=2)  # 为 True, 才能改变通道数
blk(X).shape
#TensorShape([4, 3, 3, 6])

TensorShape([4, 3, 3, 6])

## 5.11.2 ResNet模型

ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的$7\times 7$卷积层后接步幅为2的$3\times 3$的最大池化层。**不同之处在于ResNet每个卷积层后增加的批量归一化层**。

In [7]:
net = tf.keras.models.Sequential(
    [layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
    layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(pool_size=3, strides=2, padding='same')])

一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。**之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半**。

下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。

In [8]:
class ResnetBlock(tf.keras.layers.Layer):
    def __init__(self,num_channels, num_residuals, first_block=False,**kwargs):
        super(ResnetBlock, self).__init__(**kwargs)
        self.listLayers=[]
        for i in range(num_residuals):   # 残差块的个数
            if i == 0 and not first_block:
                self.listLayers.append(Residual(num_channels, use_1x1conv=True, strides=2))  # 这里的步长使得残差快，形状发生改变
            else:
                self.listLayers.append(Residual(num_channels))      # 默认 strides=1,而且不使用 1x1卷积，  第一个模块这里，不改变形状
    
    def call(self, X):
        for layer in self.listLayers.layers:
            X = layer(X)
        return X

接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。

In [9]:
class ResNet(tf.keras.Model):
    def __init__(self,num_blocks,**kwargs):   # 这里的num_blocks，[2,2,2,2]，
        super(ResNet, self).__init__(**kwargs)
        self.conv=layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn=layers.BatchNormalization()
        self.relu=layers.Activation('relu')
        self.mp=layers.MaxPool2D(pool_size=3, strides=2, padding='same')
        self.resnet_block1=ResnetBlock(64,num_blocks[0], first_block=True)   # 这里使用了 4 个模块， 每一个模块使用了2个残差块，
        self.resnet_block2=ResnetBlock(128,num_blocks[1])                    #  每个残差块里面至少两个卷积层
        self.resnet_block3=ResnetBlock(256,num_blocks[2])
        self.resnet_block4=ResnetBlock(512,num_blocks[3])
        self.gap=layers.GlobalAvgPool2D()   # 全局平均池化层
        self.fc=layers.Dense(units=10,activation=tf.keras.activations.softmax)

    def call(self, x):
        x=self.conv(x)
        x=self.bn(x)
        x=self.relu(x)
        x=self.mp(x)
        x=self.resnet_block1(x)
        x=self.resnet_block2(x)
        x=self.resnet_block3(x)
        x=self.resnet_block4(x)
        x=self.gap(x)
        x=self.fc(x)
        return x
    
mynet=ResNet([2,2,2,2])

**这个网络是真的牛逼！**

最后，与GoogLeNet一样，加入全局平均池化层后接上全连接层输出。

这里每个模块里有4个卷积层（不计算 1×1卷积层），加上最开始的卷积层和最后的全连接层，共计18层。这个模型通常也被称为ResNet-18。通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的ResNet-152。虽然ResNet的主体架构跟GoogLeNet的类似，但ResNet结构更简单，修改也更方便。这些因素都导致了ResNet迅速被广泛使用。
在训练ResNet之前，我们来观察一下输入形状在ResNet不同模块之间的变化。

In [10]:
X = tf.random.uniform(shape=(1,  224, 224 , 1))
for layer in mynet.layers:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv2d_11 output shape:	 (1, 112, 112, 64)
batch_normalization_9 output shape:	 (1, 112, 112, 64)
activation_1 output shape:	 (1, 112, 112, 64)
max_pooling2d_1 output shape:	 (1, 56, 56, 64)
resnet_block output shape:	 (1, 56, 56, 64)
resnet_block_1 output shape:	 (1, 28, 28, 128)
resnet_block_2 output shape:	 (1, 14, 14, 256)
resnet_block_3 output shape:	 (1, 7, 7, 512)
global_average_pooling2d output shape:	 (1, 512)
dense output shape:	 (1, 10)


## 5.11.3  获取数据和训练模型

下面我们在Fashion-MNIST数据集上训练ResNet。

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

mynet.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = mynet.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = mynet.evaluate(x_test, y_test, verbose=2)

Epoch 1/5
750/750 [==============================] - 92s 123ms/step - loss: 0.4767 - accuracy: 0.8311 - val_loss: 0.4067 - val_accuracy: 0.8558
Epoch 2/5
750/750 [==============================] - 99s 131ms/step - loss: 0.3270 - accuracy: 0.8810 - val_loss: 0.3340 - val_accuracy: 0.8808
Epoch 3/5
750/750 [==============================] - 103s 137ms/step - loss: 0.2793 - accuracy: 0.8972 - val_loss: 0.3020 - val_accuracy: 0.8911
Epoch 4/5
750/750 [==============================] - 103s 137ms/step - loss: 0.2567 - accuracy: 0.9058 - val_loss: 0.3032 - val_accuracy: 0.8901
Epoch 5/5
750/750 [==============================] - 103s 137ms/step - loss: 0.2419 - accuracy: 0.9104 - val_loss: 0.3204 - val_accuracy: 0.8850
313/313 - 8s - loss: 0.3207 - accuracy: 0.8818


## 小结

* 残差块通过跨层的数据通道从而能够训练出有效的深度神经网络。
* ResNet深刻影响了后来的深度神经网络的设计。

## 练习

* 参考ResNet论文的表1来实现不同版本的ResNet [1]。
* 对于比较深的网络， ResNet论文中介绍了一个“瓶颈”架构来降低模型复杂度。尝试实现它 [1]。
* 在ResNet的后续版本里，作者将残差块里的“卷积、批量归一化和激活”结构改成了“批量归一化、激活和卷积”，实现这个改进（[2]，图1）。

###  练习，待做

## 参考文献

[1] He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 770-778).

[2] He, K., Zhang, X., Ren, S., & Sun, J. (2016, October). Identity mappings in deep residual networks. In European Conference on Computer Vision (pp. 630-645). Springer, Cham.

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_convolutional-neural-networks/resnet.html)

